<a href="https://colab.research.google.com/github/Zaicevvv/GoIT-DSandML-Private/blob/main/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах.**  
  
  **Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features).**

**Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.**

Почнемо з того, що завантажимо дані з CSV-файлів, та зробимо один DataFrame, додавши до кожного вектора ознак мітку з назвою дії (класу).

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

# Папка, де зберігаються CSV-файли
data_folder = Path("/content/drive/MyDrive/data")

# Створимо DataFrame
df = pd.DataFrame(columns=['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z', 'class'])
# Наповнимо його даними у циклі
for folder in data_folder.iterdir():
    for csv_file in folder.iterdir():
        data = pd.read_csv(csv_file)
        # Додамо мітку класу
        data['class'] = folder.name
        df = pd.concat([df, data], ignore_index=True)
df

,accelerometer_X,accelerometer_Y,accelerometer_Z,class
0,17.089811,25.201366,10.448297,running
1,3.543419,16.678009,4.395755,running
2,1.446098,-5.789180,-2.193089,running
3,2.571373,8.633492,-2.820370,running
4,7.948750,14.168885,-7.192182,running
...,...,...,...,...
193855,-9.514558,-10.247183,-9.117120,walking
193856,-3.744531,-8.437167,0.167594,walking
193857,-2.614468,-2.652775,3.897761,walking
193858,-1.321599,-8.729259,-1.522712,walking


Проведемо аналіз даних, позбудемось аномалій.

In [3]:
print(df.dtypes)
print(df.isnull().sum())

# Створимо DataFrame для нормалізованих даних
df_norm = pd.DataFrame()

# Фукцію для видалення аномалій
def remove_outliers(data):
    # Якщо значення більше за мат. очікування +2 або -2 стандартних відхилення, то це аномалія.

    # Визначення верхньої та нижньої межі аномалій.
    lower_bound = data.mean() - 2 * data.std()
    upper_bound = data.mean() + 2 * data.std()

    # Видалення аномалій.
    data = data[(data >= lower_bound) & (data <= upper_bound)]

    # Заповнимо місця, де були аномалії середнім значенням по колонці
    for column in data.columns:
        data[column].fillna(data[column].mean(), inplace=True)

    return data

# Розділемо дані по класам та приберемо мітку
classes = df['class'].unique()
for class_label in classes:
    data = df[df['class'] == class_label].iloc[:,:-1]
    # Використаємо фукцію для видалення аномалій на кожному з класів
    data = remove_outliers(data)
    # Повернемо мітки на місце
    data['class'] = class_label
    # Додамо клас у новий DataFrame
    df_norm = pd.concat([df_norm, data], ignore_index=True)

df_norm

accelerometer_X    float64
accelerometer_Y    float64
accelerometer_Z    float64
class               object
dtype: object
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
class              0
dtype: int64


,accelerometer_X,accelerometer_Y,accelerometer_Z,class
0,17.089811,25.201366,10.448297,running
1,3.543419,16.678009,4.395755,running
2,1.446098,-5.789180,-2.193089,running
3,2.571373,8.633492,-2.820370,running
4,7.948750,14.168885,-7.192182,running
...,...,...,...,...
193855,-9.514558,-10.247183,-9.117120,walking
193856,-3.744531,-8.437167,0.167594,walking
193857,-2.614468,-2.652775,3.897761,walking
193858,-1.321599,-8.729259,-1.522712,walking


Тепер перемішаємо та нормалізуємо дані і створимо навчальні та тестові дані.

In [4]:
from sklearn.preprocessing import scale
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

df_shuffled = shuffle(df_norm, random_state=42)
X = scale(df_shuffled[df_shuffled.columns[:-1]])
y = df_shuffled['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Перейдемо до навчання моделей SVM та випадкового лісу.

In [ ]:
import time
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Навчання моделі SVM
start_time = time.time()
svm_model = SVC(kernel='linear').fit(X_train, y_train)
svm_training_time = time.time() - start_time

# Навчання моделі випадкового лісу
start_time = time.time()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)
rf_training_time = time.time() - start_time

# Оцінка результатів
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# Отримання звіту про класифікацію
svm_report = classification_report(y_test, svm_predictions)
rf_report = classification_report(y_test, rf_predictions)

print("SVM classification report:")
print("Training time:", svm_training_time, "seconds")
print(svm_report)

print("\nRandom Forest classification report:")
print("Training time:", rf_training_time, "seconds")
print(rf_report)

Змінимо датасет. Розрахуємо часові ознаки (time domain features).

In [8]:
# Функція для витягнення часових ознак з акселерометра
def extract_features(data):
    features = []
    # Додамо середнє значення для кожного стовпця
    features.extend(np.mean(data, axis=0))
    # Додамо середнє квадратичне відхилення для кожного стовпця
    features.extend(np.std(data, axis=0))
    # Додамо медіану для кожного стовпця
    features.extend(np.median(data, axis=0))
    # Додамо мінімальне та максимальне значення для кожного стовпця
    features.extend(np.min(data, axis=0))
    features.extend(np.max(data, axis=0))
    return features

In [ ]:




# # Навчання моделі SVM
# start_time = time.time()
# svm_model_processed = SVC(kernel='linear')
# svm_model_processed.fit(X_train_processed_scaled, y_train_processed)
# svm_training_time = time.time() - start_time

# # Навчання моделі випадкового лісу
# start_time = time.time()
# rf_model_processed = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model_processed.fit(X_train_processed_scaled, y_train_processed)
# rf_training_time = time.time() - start_time

# # Виведемо час навчання для кожної моделі
# print("SVM classification report:")
# print("Training time:", svm_training_time, "seconds")
# print(svm_report)

# print("\nRandom Forest classification report:")
# print("Training time:", rf_training_time, "seconds")
# print(rf_report)

У підсумку можна сказати, що випадковий ліс показав ідеальні результати на тестових даних, на відміну від SVM.